#Load Library

In [1]:
import math
import pandas as pd
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from scipy.stats import norm
from dataclasses import dataclass

#Creating Dataset

In [2]:
#  Generating random dataset with n = 1000 (# of data points) and d = 2 (# of dimensions)
x1 = np.random.randint(2, size = 1000)
x2 = np.random.randint(2, size = 1000)
y = np.random.randint(2, size = 1000)

df_sample = pd.DataFrame({'x1': x1, 'x2': x2, 'y': y})
df_sample

,x1,x2,y
0,0,0,1
1,0,0,0
2,1,1,0
3,0,1,0
4,0,1,1
...,...,...,...
995,1,0,1
996,1,0,1
997,0,0,1
998,0,0,1


In [3]:
#creating an array for discrete generated variables
X= np.vstack((x1,x2)).T

In [4]:
X

array([[0, 0],
       [0, 0],
       [1, 1],
       ...,
       [0, 0],
       [0, 0],
       [0, 0]])

In [5]:
@dataclass
class DiscreteNaiveBayes:
  X: None
  y: None
  laplace: int = 0

  def split(self):
    self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(self.X, self.y, test_size = 0.1, shuffle=True)

  def probability(self, X, prior, dist1, dist2):
    return prior * dist1 * dist2

  def fit(self):
    self.split()
    self.X_y0_train = self.X_train[self.y_train == 0] # all training records with y=0
    self.X_y1_train = self.X_train[self.y_train == 1] # all training records with y=1
    self.X1_y0_train = self.X_y0_train[:, 0]
    self.X2_y0_train = self.X_y0_train[:, 1]
    self.X1_y1_train = self.X_y1_train[:, 0]
    self.X2_y1_train = self.X_y1_train[:, 1]

    # Calculating likelihoods

    # class - 0
    self.X10_y0_train = self.X1_y0_train[self.X1_y0_train == 0]
    self.X11_y0_train = self.X1_y0_train[self.X1_y0_train == 1]
    self.X20_y0_train = self.X2_y0_train[self.X2_y0_train == 0]
    self.X21_y0_train = self.X2_y0_train[self.X2_y0_train == 1]

    # class - 1
    self.X10_y1_train = self.X1_y1_train[self.X1_y1_train == 0]
    self.X11_y1_train = self.X1_y1_train[self.X1_y1_train == 1]
    self.X20_y1_train = self.X2_y1_train[self.X2_y1_train == 0]
    self.X21_y1_train = self.X2_y1_train[self.X2_y1_train == 1]

    # priors for y
    self.prior_y0 = len(self.X_y0_train)/len(self.X_train)
    self.prior_y1 = len(self.X_y1_train)/len(self.X_train)

    #k is the number of features in dataset
    k = 2
    laplace = self.laplace
    print(laplace)

    # calculating likelihood functions for x0 and x1 - class 0
    self.likelihood_X10_y0 = (len(self.X10_y0_train)+laplace)/(len(self.X_y0_train)+(k*laplace))
    self.likelihood_X11_y0 = (len(self.X11_y0_train)+laplace)/(len(self.X_y0_train)+(k*laplace))
    self.likelihood_X20_y0 = (len(self.X20_y0_train)+laplace)/(len(self.X_y0_train)+(k*laplace))
    self.likelihood_X21_y0 = (len(self.X21_y0_train)+laplace)/(len(self.X_y0_train)+(k*laplace))

    # calculating likelihood functions for x0 and x1 - class 1
    self.likelihood_X10_y1 = (len(self.X10_y1_train)+laplace)/(len(self.X_y1_train)+(k*laplace))
    self.likelihood_X11_y1 = (len(self.X11_y1_train)+laplace)/(len(self.X_y1_train)+(k*laplace))
    self.likelihood_X20_y1 = (len(self.X20_y1_train)+laplace)/(len(self.X_y1_train)+(k*laplace))
    self.likelihood_X21_y1 = (len(self.X21_y1_train)+laplace)/(len(self.X_y1_train)+(k*laplace))

  def predict(self):
    self.y_pred = []
    self.y_true = []

    for sample, target in zip(self.X_test, self.y_test):
        # if sample belongs to class 0
        if sample[0] == 1:
          self.dist_X1_y0 = self.likelihood_X11_y0 #X1=1|y=0
        else:
          self.dist_X1_y0 = self.likelihood_X10_y0 #X1=0|y=0
        if sample[1] == 1:
          self.dist_X2_y0 = self.likelihood_X21_y0 #X2=1|y=0
        else:
          self.dist_X2_y0 = self.likelihood_X20_y0 #X2=0|y=0

        # if sample belongs to class 1
        if sample[0] == 1:
            self.dist_X1_y1 = self.likelihood_X11_y1 #X1=1|y=1
        else:
            self.dist_X1_y1 = self.likelihood_X10_y1 #X1=0|y=1
        if sample[1] == 1:
            self.dist_X2_y1 = self.likelihood_X21_y1 #X2=1|y=1
        else:
            self.dist_X2_y1 = self.likelihood_X20_y1 #X2=0|y=1

        py0 = self.probability(sample, self.prior_y0, self.dist_X1_y0, self.dist_X2_y0)
        py1 = self.probability(sample, self.prior_y1, self.dist_X1_y1, self.dist_X2_y1)

        predicted_class = np.argmax([py0, py1])

        print(f'P(y=0| {sample}) = {py0:.4f}')
        print(f'P(y=1| {sample}) = {py1:.4f}')
        print(f'Model predicted class {predicted_class} and the truth was: {target}\n')

        self.y_pred.append(predicted_class)
        self.y_true.append(target)

    print("Predicted Class: ",self.y_pred)
    print("Actual Class: ",self.y_true)
    accuracy, precision, recall, f1_score = self.evaluate(self.y_pred, self.y_true)

    print("Accuracy: ", round(accuracy*100,2))
    print("Precision: ", round(precision*100,2))
    print('Recall: ', round(recall*100,2))
    print("F1_score: ", round(f1_score*100,2))

  def evaluate(self,y_true, y_pred):
    y = np.array(y_true)
    y_pred = np.array(y_pred)
    y = (y==1).astype(int)
    y_pred = (y_pred ==1).astype(int)

    accuracy= (y==y_pred).sum()/len(y)
    precision = (y & y_pred).sum()/y_pred.sum()
    recall = (y*y_pred).sum()/y.sum()

    f1_score = 2*(precision*recall)/(precision+recall)

    y_pred = (y_pred == 1)

    return accuracy, precision, recall, f1_score


Without Laplace Smoothing

In [9]:
nb = DiscreteNaiveBayes(X, y)
nb.fit()
nb.predict()

0
P(y=0| [1 1]) = 0.1306
P(y=1| [1 1]) = 0.1264
Model predicted class 0 and the truth was: 0

P(y=0| [1 0]) = 0.1172
P(y=1| [1 0]) = 0.1369
Model predicted class 1 and the truth was: 1

P(y=0| [1 1]) = 0.1306
P(y=1| [1 1]) = 0.1264
Model predicted class 0 and the truth was: 0

P(y=0| [0 1]) = 0.1172
P(y=1| [0 1]) = 0.1280
Model predicted class 1 and the truth was: 1

P(y=0| [0 0]) = 0.1051
P(y=1| [0 0]) = 0.1386
Model predicted class 1 and the truth was: 0

P(y=0| [0 0]) = 0.1051
P(y=1| [0 0]) = 0.1386
Model predicted class 1 and the truth was: 0

P(y=0| [1 1]) = 0.1306
P(y=1| [1 1]) = 0.1264
Model predicted class 0 and the truth was: 1

P(y=0| [0 1]) = 0.1172
P(y=1| [0 1]) = 0.1280
Model predicted class 1 and the truth was: 0

P(y=0| [1 1]) = 0.1306
P(y=1| [1 1]) = 0.1264
Model predicted class 0 and the truth was: 0

P(y=0| [1 0]) = 0.1172
P(y=1| [1 0]) = 0.1369
Model predicted class 1 and the truth was: 0

P(y=0| [0 0]) = 0.1051
P(y=1| [0 0]) = 0.1386
Model predicted class 1 and the 

With Laplace Smoothing

In [10]:
nb = DiscreteNaiveBayes(X, y, laplace=1)
nb.fit()
nb.predict()

1
P(y=0| [1 1]) = 0.1265
P(y=1| [1 1]) = 0.1324
Model predicted class 1 and the truth was: 0

P(y=0| [0 0]) = 0.1077
P(y=1| [0 0]) = 0.1335
Model predicted class 1 and the truth was: 1

P(y=0| [1 1]) = 0.1265
P(y=1| [1 1]) = 0.1324
Model predicted class 1 and the truth was: 0

P(y=0| [0 0]) = 0.1077
P(y=1| [0 0]) = 0.1335
Model predicted class 1 and the truth was: 1

P(y=0| [1 0]) = 0.1168
P(y=1| [1 0]) = 0.1409
Model predicted class 1 and the truth was: 0

P(y=0| [0 1]) = 0.1168
P(y=1| [0 1]) = 0.1254
Model predicted class 1 and the truth was: 0

P(y=0| [1 1]) = 0.1265
P(y=1| [1 1]) = 0.1324
Model predicted class 1 and the truth was: 1

P(y=0| [0 1]) = 0.1168
P(y=1| [0 1]) = 0.1254
Model predicted class 1 and the truth was: 0

P(y=0| [0 1]) = 0.1168
P(y=1| [0 1]) = 0.1254
Model predicted class 1 and the truth was: 0

P(y=0| [0 0]) = 0.1077
P(y=1| [0 0]) = 0.1335
Model predicted class 1 and the truth was: 1

P(y=0| [1 1]) = 0.1265
P(y=1| [1 1]) = 0.1324
Model predicted class 1 and the 

We can see that by implementing laplace smoothing we get better accuracy, precision, recall and f1 score.